### HW04: Practice with feature engineering, splitting data, and fitting and regularizing linear models

Kara Conrad, 9082326472

### Hello Students:

- Start by downloading HW04.ipynb from this folder. Then develop it into your solution.
- Write code where you see "... your code here ..." below.
  (You are welcome to use more than one cell.)
- If you have questions, please ask them in class, office hours, or piazza. Our TA
  and I are very happy to help with the programming (provided you start early
  enough, and provided we are not helping so much that we undermine your learning).
- When you are done, run these Notebook commands:
  - Shift-L (once, so that line numbers are visible)
  - Kernel > Restart and Run All (run all cells from scratch)
  - Esc S (save)
  - File > Download as > HTML
- Turn in:
  - HW04.ipynb to Canvas's HW04.ipynb assignment
  - HW04.html to Canvas's HW04.html assignment
  - As a check, download your files from Canvas to a new 'junk' folder. Try 'Kernel > Restart
  and Run All' on the '.ipynb' file to make sure it works. Glance through the '.html' file.
- Turn in partial solutions to Canvas before the deadline. e.g. Turn in part 1,
  then parts 1 and 2, then your whole solution. That way we can award partial credit
  even if you miss the deadline. We will grade your last submission before the deadline.

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn import linear_model

## 1. Feature engineering (one-hot encoding and data imputation)

(Note: This paragraph is not instructions but rather is to communicate context for this exercise. We use the same Titanic data we used in HW02:
- There we used `df.dropna()` to drop any observations with missing values; here we use data imputation instead.
- There we manually did one-hot encoding of the categorical `Sex` column by making a `Female` column; here we do the same one-hot encoding with the help of pandas's `df.join(pd.get_dummies())`.
- There we used a decision tree; here we use $k$-NN.

We evaluate how these strategies can improve model performance by allowing us to use columns with categorical or missing data.)

### 1a. Read the data from [http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv](http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv).
- Retain only these columns: Survived, Pclass, Sex, Age, SibSp, Parch.
- Display the first 7 rows.

These data are described at [https://www.kaggle.com/competitions/titanic/data](https://www.kaggle.com/competitions/titanic/data) (click on the small down-arrow to see the "Data Dictionary"), which is where they are from.
- Read that "Data Dictionary" paragraph (with your eyes, not python) so you understand what each column represents.


In [2]:
df = pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv")[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch"]]
df.head(n=7)

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0
5,0,3,male,NaN,0,0
6,0,1,male,54.0,0,0


### 1b. Try to train a $k$NN model to predict $y=$ 'Survived' from $X=$ these features: 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'.
- Use $k = 3$ and the (default) euclidean metric.
- Notice at the bottom of the error message that it fails with the error "ValueError: could not convert string to float: 'male'".
- Comment out your .fit() line so the cell can run without error.

In [3]:
kNN = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
y=df["Survived"]
X=df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']]
#kNN.fit(X, y)

### 1c. Try to train again, this time without the 'Sex' feature.
- Notice that it fails because "Input contains NaN".
- Comment out your .fit() line so the cell can run without error.
- Run `X.isna().any()` (where X is the name of your DataFrame of features) to see that
  the 'Age' feature has missing values. (You can see the first missing value in
  the sixth row that you displayed above.)

In [4]:
X=df[['Pclass', 'Age', 'SibSp', 'Parch']]
#kNN.fit(X, y)
X.isna().any()

Pclass    False
Age        True
SibSp     False
Parch     False
dtype: bool

### 1d. Train without the 'Sex' and 'Age' features.
- Report accuracy on the training data with a line of the form
  `Accuracy on training data is  0.500` (0.500 may not be correct).

In [5]:
X=df[['Pclass', 'SibSp', 'Parch']]
kNN.fit(X, y)
print(f'Accuracy on training data is {round(kNN.score(X, y), 3)}.')

Accuracy on training data is 0.633.


### 1e.  Use one-hot encoding
to include a binary 'male'  feature made from the 'Sex' feature. (Or include a binary 'female'
feature, according to your preference. Using both is unnecessary since either is the logical
negation of the other.) That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male'.
- Use pandas's df.join(pd.get_dummies())`.
- Report training accuracy as before.

In [6]:
X=df[['Pclass', 'Sex', 'SibSp', 'Parch']]

X=X.join(pd.get_dummies(X["Sex"])["male"])
X=X.drop(["Sex"], axis=1)

kNN.fit(X, y)

print(f'Accuracy on training data is {round(kNN.score(X, y), 3)}.')

Accuracy on training data is 0.744.


### 1f. Use data imputation
to include an 'age' feature made from 'Age' but replacing each missing value with the median
of the non-missing ages. That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male',
'age'.

- Report training accuracy as before.

In [7]:
X2 = X.join(df["Age"])
X2["Age"] = X2["Age"].fillna(X2["Age"].median())
kNN.fit(X2, y)
print(f'Accuracy on training data is {round(kNN.score(X2, y), 3)}.')

Accuracy on training data is 0.861.


## 2. Explore model fit, overfit, and regularization in the context of multiple linear regression

### 2a. Prepare the data:
- Read [http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv](http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv) into a DataFrame.
- Set a variable `X` to the subset consisting of all columns except `mpg`.
- Set a variable `y` to the `mpg` column.
- Use `train_test_split()` to split `X` and `y` into `X_train`, `X_test`, `y_train`, and `y_test`.
  - Reserve half the data for training and half for testing.
  - Use `random_state=0` to get reproducible results.

In [4]:
dff = pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv")
#dff = dff.set_index("Unnamed: 0")
X = dff.drop("mpg", axis=1)
X = pd.get_dummies(X, drop_first=True)
y = dff["mpg"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

### 2b. Train three models on the training data and evaluate each on the test data:
- `LinearRegression()`
- `Lasso()`
- `Ridge()`

The evaluation consists in displaying MSE$_\text{train}, $ MSE$_\text{test}$, and the coefficients $\mathbf{w}$ for each model.

In [9]:
models = [linear_model.LinearRegression(),
              linear_model.Lasso(), #sets some coeffs to zero
              linear_model.Ridge()]

for model in models:
        model.fit(X_train, y_train)
        MSE_train = (1/y_train.size) * np.sum((y_train - model.predict(X_train))**2)
        MSE_test = (1/y_test.size)  * np.sum((y_test - model.predict(X_test))**2)
        
        print("MSE_tr:",MSE_train, "\nMSE_te:",MSE_test, "\nCoeffs:", model.coef_, "\n")

MSE_tr: 5.443140246024981e-29 
MSE_te: 11.198177807346006 
Coeffs: [-1.39244595e-02 -7.44628132e-03  4.00323924e-03  7.92716848e-01
 -3.21854592e+00  1.09825372e+00 -4.82369921e-01  4.70797947e-01
  1.49546846e+00 -1.04547895e+00  4.44089210e-16 -3.91445660e-02
 -2.22044605e-16  0.00000000e+00 -1.65252552e+00  1.06274512e-01
  2.60480513e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.32319472e-01 -6.87968731e-01  1.02875601e+00  1.03011212e+00
  1.40754216e+00  0.00000000e+00 -4.22170331e-01 -2.16463206e-01
  0.00000000e+00  1.85342041e-02 -6.61886365e-01  0.00000000e+00
  0.00000000e+00  3.87050385e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  2.75586445e+00  0.00000000e+00  0.00000000e+00
 -3.94677511e+00] 

MSE_tr: 5.692738046538883 
MSE_te: 12.99192397981821 
Coeffs: [-0.         -0.03718773 -0.01681757  0.         -0.          0.
  0.          0.          0.         -0.84712891  0.          0.
  0.          0.         -0.          0.         -0.          0.
  0.  

### 2c. Answer a few questions about the models:
- Which one best fits the training data?
- Which one best fits the test data?
- Which one does feature selection by setting most coefficients to zero?- 

 - The lower the MSE value is the better the models fit is because there is less cumulative error between the actually and predicted y values. For the Linear Regression model the MSE value for the training data is the lowest of the three models.
 - For the test date the best fit model is the Ridge model since it has the lowerest MSE value.
 - The Lasso model does feature selection by setting most coefficients to zero.